# Clustering Neighbourhoods in Toronto 

## Part 1

### Data Scraping

In [1]:
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
pip install lxml html5lib beautifulsoup4

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.0/libexec/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
dfs = pd.read_html(url)

In [5]:
df = dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
df.shape

(180, 3)

### Data Cleaning

In [7]:
df1 = df[df.Borough != 'Not assigned']
df1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df1.groupby('Postal Code')
df1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df1.loc[df1['Neighbourhood'] == 'Not assigned']     

,Postal Code,Borough,Neighbourhood


In [10]:
for bor, neigh in zip(df1['Borough'],df1['Neighbourhood']):
    
    if neigh == 'Not assigned':
        print('not assigned')
        df1['Neighbourhood'].replace({neigh:bor})
        

It is given that if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. But as per the above two codes(Input[8] and Input[9]), it is clear that no such neighborhoods exist.

In [11]:
df_cleaned = df1.reset_index(drop=True)
df_cleaned.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
df_cleaned.shape

(103, 3)

## Part 2

### Constructing a pandas dataframe with the Geospatial data for each postal code

In [13]:
df_loc = pd.read_csv('http://cocl.us/Geospatial_data')
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_merged = pd.merge(df_cleaned, df_loc, on="Postal Code", how="left")
df_merged.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3

In [15]:
import sys
!{sys.executable} -m pip install geopy
import geopy
from geopy.geocoders import Nominatim

import json 

import requests 

from pandas.io.json import json_normalize

!{sys.executable} -m pip install matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

!{sys.executable} -m pip install folium
import folium

print('Libraries imported.')

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.0/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.0/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.0/libexec/bin/python3.8 -m pip install --upgrade pip' command.
Libraries imported.


#### Getting only the Torornto neighbourhoods by considering the boroughs that contain the word Toronto 

In [16]:
df_toronto = df_merged[df_merged['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [17]:
df_toronto['Borough'].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

In [18]:
df_DT = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
df_DT.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### Mapping the Toronto Neighbourhoods(Postal Codes)

In [19]:
address = 'Toronto, T'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, postal in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Postal Code']):
    label = '{}, {}'.format(postal, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Explore around each postal code using Foursquare

In [21]:
CLIENT_ID ='????' 
CLIENT_SECRET ='????' 
VERSION ='????'

In [22]:
First_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
First_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

First_postal_code = df_toronto.loc[0, 'Postal Code'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(First_postal_code, 
                                                               First_latitude,First_longitude))

Latitude and longitude values of M5A are 43.6542599, -79.3606359.


In [23]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    First_latitude, 
    First_longitude, 
    radius, 
    LIMIT)

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f31275856e24d6083c4d4fd'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

#### Get the venue categories around each postal code

In [25]:
def getNearbyVenues(postal_code, latitudes, longitudes, radius=500):
    
    column_names = ['Postal Code','Latitude', 'Longitude','Venue Category']
    nearby_venues = pd.DataFrame(columns=column_names)
    
    i = 0
    for post, lat, lng in zip(postal_code, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        items = requests.get(url).json()["response"]['groups'][0]['items']
           
        for item in items:
            venue_category = item['venue']['categories'][0]['name']
            
            venue_list = []
            venue_list.append(post)
            venue_list.append(lat)
            venue_list.append(lng)
            venue_list.append(venue_category)
            nearby_venues.loc[i] = venue_list
            i = i + 1
                             
   
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(postal_code = df_toronto['Postal Code'],
                                   latitudes= df_toronto['Latitude'],
                                   longitudes= df_toronto['Longitude'])

In [27]:
toronto_venues.head()

,Postal Code,Latitude,Longitude,Venue Category
0,M5A,43.65426,-79.360636,Bakery
1,M5A,43.65426,-79.360636,Coffee Shop
2,M5A,43.65426,-79.360636,Distribution Center
3,M5A,43.65426,-79.360636,Spa
4,M5A,43.65426,-79.360636,Restaurant


In [28]:
toronto_venues['Venue Category'].unique()

array(['Bakery', 'Coffee Shop', 'Distribution Center', 'Spa',
       'Restaurant', 'Park', 'Pub', 'Historic Site', 'Breakfast Spot',
       'Gym / Fitness Center', 'Farmers Market', 'Chocolate Shop',
       'Dessert Shop', 'Performing Arts Venue', 'Theater',
       'French Restaurant', 'Café', 'Mexican Restaurant', 'Yoga Studio',
       'Event Space', 'Shoe Store', 'Ice Cream Shop', 'Art Gallery',
       'Cosmetics Shop', 'Electronics Store', 'Bank', 'Beer Store',
       'Hotel', 'Antique Shop', 'Portuguese Restaurant',
       'Persian Restaurant', 'Creperie', 'Beer Bar',
       'Arts & Crafts Store', 'Sushi Restaurant', 'Hobby Shop',
       'Japanese Restaurant', 'Diner', 'Fried Chicken Joint',
       'Smoothie Shop', 'Sandwich Place', 'College Auditorium', 'Gym',
       'Bar', 'College Cafeteria', 'Vegetarian / Vegan Restaurant',
       'Clothing Store', 'Comic Shop', 'Plaza', 'Pizza Place',
       'Burrito Place', 'Music Venue', 'Burger Joint', 'Thai Restaurant',
       'Movie Theat

#### Getting the top most common venues for each potal code

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot
# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
toronto_s = toronto_onehot.groupby('Postal Code').sum().reset_index()
toronto_s.head()

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4K,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,M4L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4M,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,1,0,0,1
4,M4N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
import numpy as np

In [32]:
def return_most_common_venues(row, num_top_venues):
    
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]    

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted
neighborhoods_venues_sorted['Postal Code'] = toronto_s['Postal Code']

for ind in np.arange(toronto_s.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_s.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Trail,Pub,Neighborhood,Health Food Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Indian Restaurant,Spa,Caribbean Restaurant
2,M4L,Park,Pizza Place,Sandwich Place,Sushi Restaurant,Ice Cream Shop,Fish & Chips Shop,Fast Food Restaurant,Italian Restaurant,Liquor Store,Movie Theater
3,M4M,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Yoga Studio,Neighborhood,Sandwich Place,Cheese Shop
4,M4N,Park,Bus Line,Swim School,Dim Sum Restaurant,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Clustering neighbourhoods(Postal Codes)

In [34]:
!{sys.executable} -m pip install sklearn  
from sklearn.cluster import KMeans

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.0/libexec/bin/python3.8 -m pip install --upgrade pip' command.


In [37]:
# set number of clusters
kclusters = 5

toronto_clustering = toronto_s.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 
kmeans

KMeans(n_clusters=5, random_state=0)

In [46]:
#add clustering labels
df2 = neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

toronto_merged = pd.merge(df_loc, neighborhoods_venues_sorted, on="Postal Code")
toronto_merged.head(10)


,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,43.676357,-79.293031,1,Trail,Pub,Neighborhood,Health Food Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio
1,M4K,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Indian Restaurant,Spa,Caribbean Restaurant
2,M4L,43.668999,-79.315572,1,Park,Pizza Place,Sandwich Place,Sushi Restaurant,Ice Cream Shop,Fish & Chips Shop,Fast Food Restaurant,Italian Restaurant,Liquor Store,Movie Theater
3,M4M,43.659526,-79.340923,0,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Yoga Studio,Neighborhood,Sandwich Place,Cheese Shop
4,M4N,43.728020,-79.388790,1,Park,Bus Line,Swim School,Dim Sum Restaurant,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,M4P,43.712751,-79.390197,1,Gym,Hotel,Park,Sandwich Place,Food & Drink Shop,Department Store,Breakfast Spot,Pizza Place,Gym / Fitness Center,Concert Hall
6,M4R,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Yoga Studio,Salon / Barbershop,Restaurant,Mexican Restaurant,Metro Station,Fast Food Restaurant,Chinese Restaurant,Sporting Goods Shop
7,M4S,43.704324,-79.388790,0,Dessert Shop,Sandwich Place,Pizza Place,Sushi Restaurant,Café,Coffee Shop,Gym,Italian Restaurant,Farmers Market,Diner
8,M4T,43.689574,-79.383160,1,Park,Restaurant,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
9,M4V,43.686412,-79.400049,1,Pub,Coffee Shop,Restaurant,Bank,Supermarket,Sushi Restaurant,Sports Bar,Fried Chicken Joint,Pizza Place,Bagel Shop


In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, pos, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(pos) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters